In [1]:
import os
import sys
import time 
import random
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import sum as spark_sum, avg, col, round
from pyspark.sql.functions import when, isnan, count, rand, expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, rank
from pyspark.ml.feature import Imputer



In [2]:
# Configuration pour Windows
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['HADOOP_HOME'] = os.path.abspath('.')


In [3]:
spark = SparkSession.builder \
    .appName("SimpleApp") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.python.worker.timeout", "300") \
    .config("spark.network.timeout", "300s") \
    .getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
rdd_squared = rdd.map(lambda x: x ** 2)
print(rdd_squared.collect())  # Result: [1, 4, 9, 16, 25]

# Stop SparkSession


[1, 4, 9, 16, 25]


EXERCICE 1 

In [6]:


# Create an RDD from a list of integers
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd = sc.parallelize(data)

# 1. Filter even numbers
even_rdd = rdd.filter(lambda x: x % 2 == 0)
print("Even numbers:", even_rdd.collect())

# 2. Calculate square of each filtered number
squared_rdd = even_rdd.map(lambda x: x ** 2)
print("Squares of even numbers:", squared_rdd.collect())

# 3. Calculate the sum
sum_result = squared_rdd.reduce(lambda x, y: x + y)
print("Sum of squares of even numbers:", sum_result)



Even numbers: [2, 4, 6, 8, 10]
Squares of even numbers: [4, 16, 36, 64, 100]
Sum of squares of even numbers: 220


EXERCICE 2

In [7]:
employee_data = [
    ("John Doe", "IT", 65000),
    ("Jane Smith", "HR", 45000),
    ("Robert Johnson", "Finance", 75000),
    ("Maria Garcia", "IT", 55000),
    ("James Brown", "Marketing", 48000),
    ("Emily Davis", "Finance", 62000),
    ("Michael Wilson", "HR", 51000),
    ("Sarah Thompson", "Marketing", 49000)
]

# Define schema
schema = StructType([
    StructField("name", StringType(), nullable=False),
    StructField("department", StringType(), nullable=False),
    StructField("salary", IntegerType(), nullable=False)
])

In [8]:
df_employees = spark.createDataFrame(employee_data, schema=schema)

# 1. Display the schema
print("DataFrame Schema:")
df_employees.printSchema()

# 2. Filter employees with salary > 50000
high_salary_df = df_employees.filter(col("salary") > 50000)
print("\nEmployees with salary > 50000:")
high_salary_df.show()

# 3. Calculate average salary by department
avg_salary_by_dept = df_employees.groupBy("department").agg(avg("salary").alias("avg_salary"))
print("\nAverage Salary by Department:")
avg_salary_by_dept.show()

DataFrame Schema:
root
 |-- name: string (nullable = false)
 |-- department: string (nullable = false)
 |-- salary: integer (nullable = false)


Employees with salary > 50000:
+--------------+----------+------+
|          name|department|salary|
+--------------+----------+------+
|      John Doe|        IT| 65000|
|Robert Johnson|   Finance| 75000|
|  Maria Garcia|        IT| 55000|
|   Emily Davis|   Finance| 62000|
|Michael Wilson|        HR| 51000|
+--------------+----------+------+


Average Salary by Department:
+----------+----------+
|department|avg_salary|
+----------+----------+
|        IT|   60000.0|
|        HR|   48000.0|
|   Finance|   68500.0|
| Marketing|   48500.0|
+----------+----------+



EXERCICE 3 

In [9]:
products_df = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("products.csv")

# 1. Afficher les 5 premières lignes
print("Les 5 premières lignes du DataFrame:")
products_df.show(5)

# 2. Filtrer les produits d'une catégorie spécifique (Electronics)
electronics_df = products_df.filter(col("category") == "Electronics")
print("\nProduits de la catégorie Electronics:")
electronics_df.show()

# 3. Calculer le prix moyen par catégorie
avg_price_by_category = products_df.groupBy("category") \
                                  .agg(avg("price").alias("prix_moyen"))
print("\nPrix moyen par catégorie:")
avg_price_by_category.show()

Les 5 premières lignes du DataFrame:
+---+--------------------+-----------+-------+
| id|                name|   category|  price|
+---+--------------------+-----------+-------+
|  1|  Laptop Dell XPS 15|Electronics|1299.99|
|  2|       iPhone 13 Pro|Electronics| 999.99|
|  3|Chaise de bureau ...|  Furniture|  249.5|
|  4|Table de salle à ...|  Furniture| 399.95|
|  5|Monitor LG UltraWide|Electronics| 349.99|
+---+--------------------+-----------+-------+
only showing top 5 rows


Produits de la catégorie Electronics:
+---+--------------------+-----------+-------+
| id|                name|   category|  price|
+---+--------------------+-----------+-------+
|  1|  Laptop Dell XPS 15|Electronics|1299.99|
|  2|       iPhone 13 Pro|Electronics| 999.99|
|  5|Monitor LG UltraWide|Electronics| 349.99|
|  8|Casque audio sans...|Electronics| 199.95|
| 10|Tablette Samsung ...|Electronics| 499.99|
| 12|  Enceinte Bluetooth|Electronics|  79.99|
+---+--------------------+-----------+-------+


Prix

EXERCICE 4

In [10]:
products_df = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("products.csv")

# Afficher le DataFrame original
print("DataFrame original:")
products_df.show()

# Définir un seuil de prix
PRICE_THRESHOLD = 300.0

# Définir une fonction Python pour classifier les prix
def classify_price(price):
    if price > PRICE_THRESHOLD:
        return "Élevé"
    else:
        return "Bas"

# Enregistrer la fonction comme UDF (User Defined Function)
price_classifier_udf = udf(classify_price, StringType())

# Appliquer l'UDF au DataFrame pour créer une nouvelle colonne
products_with_classification = products_df.withColumn(
    "price_category", 
    price_classifier_udf(col("price"))
)

# Afficher le DataFrame avec la nouvelle colonne
print("\nDataFrame avec classification des prix (seuil = {}):".format(PRICE_THRESHOLD))
products_with_classification.show()

# Compter le nombre de produits par catégorie de prix
print("\nRépartition des produits par catégorie de prix:")
products_df = spark.read.option("header", "true") \
                        .option("inferSchema", "true") \
                        .csv("products.csv")

# Afficher le DataFrame original
print("DataFrame original:")
products_df.show()

# Définir un seuil de prix
PRICE_THRESHOLD = 300.0

# Définir une fonction Python pour classifier les prix
def classify_price(price):
    if price > PRICE_THRESHOLD:
        return "Élevé"
    else:
        return "Bas"

# Enregistrer la fonction comme UDF (User Defined Function)
price_classifier_udf = udf(classify_price, StringType())

# Appliquer l'UDF au DataFrame pour créer une nouvelle colonne
products_with_classification = products_df.withColumn(
    "price_category", 
    price_classifier_udf(col("price"))
)

# Afficher le DataFrame avec la nouvelle colonne
print("\nDataFrame avec classification des prix (seuil = {}):".format(PRICE_THRESHOLD))
products_with_classification.show()

# Compter le nombre de produits par catégorie de prix
print("\nRépartition des produits par catégorie de prix:")

DataFrame original:
+---+--------------------+-----------+-------+
| id|                name|   category|  price|
+---+--------------------+-----------+-------+
|  1|  Laptop Dell XPS 15|Electronics|1299.99|
|  2|       iPhone 13 Pro|Electronics| 999.99|
|  3|Chaise de bureau ...|  Furniture|  249.5|
|  4|Table de salle à ...|  Furniture| 399.95|
|  5|Monitor LG UltraWide|Electronics| 349.99|
|  6| Lampe de bureau LED|       Home|  59.99|
|  7|      Canapé en cuir|  Furniture|  899.0|
|  8|Casque audio sans...|Electronics| 199.95|
|  9|      Tapis de salon|       Home|  129.5|
| 10|Tablette Samsung ...|Electronics| 499.99|
| 11|Bureau en bois ma...|  Furniture|  549.0|
| 12|  Enceinte Bluetooth|Electronics|  79.99|
| 13| Plante artificielle|       Home|  39.95|
| 14|Cafetière automat...|    Kitchen| 149.99|
| 15|      Lit queen size|  Furniture|  799.0|
+---+--------------------+-----------+-------+


DataFrame avec classification des prix (seuil = 300.0):
+---+--------------------+---

EXERCICE 5 : Jointure de Deux DataFrames

In [11]:
# Charger les données des commandes
orders_df = spark.read.option("header", "true") \
                      .option("inferSchema", "true") \
                      .csv("orders.csv")

# Charger les données des clients
customers_df = spark.read.option("header", "true") \
                         .option("inferSchema", "true") \
                         .csv("customers.csv")


# Afficher les DataFrames originaux
print("DataFrame des commandes:")
orders_df.show()

print("\nDataFrame des clients:")
customers_df.show()

# Effectuer une jointure interne (INNER JOIN)
inner_join_df = orders_df.join(
    customers_df,
    orders_df["customer_id"] == customers_df["customer_id"],
    "inner"
)

# Sélectionner les colonnes pertinentes
result_df = inner_join_df.select(
    "order_id",
    "amount",
    "order_date",
    customers_df["customer_id"],
    "name",
    "country"
)

# Afficher le résultat de la jointure interne
print("\nRésultat de la jointure interne:")
result_df.show()

# Effectuer une jointure externe gauche (LEFT JOIN)
# Pour inclure les commandes sans correspondance de client
left_join_df = orders_df.join(
    customers_df,
    orders_df["customer_id"] == customers_df["customer_id"],
    "left"
).select(
    "order_id",
    "amount",
    "order_date",
    orders_df["customer_id"],
    "name",
    "country"
)

# Afficher le résultat de la jointure gauche
print("\nRésultat de la jointure gauche (LEFT JOIN):")
left_join_df.show()

# Effectuer une jointure externe droite (RIGHT JOIN)
# Pour inclure les clients sans commandes
right_join_df = orders_df.join(
    customers_df,
    orders_df["customer_id"] == customers_df["customer_id"],
    "right"
).select(
    "order_id",
    "amount",
    "order_date",
    customers_df["customer_id"],
    "name", 
    "country"
)

# Afficher le résultat de la jointure droite
print("\nRésultat de la jointure droite (RIGHT JOIN):")
right_join_df.show()


DataFrame des commandes:
+--------+-----------+------+-------------------+
|order_id|customer_id|amount|         order_date|
+--------+-----------+------+-------------------+
|    1001|        101| 150.5|2023-01-15 00:00:00|
|    1002|        103|299.99|2023-01-16 00:00:00|
|    1003|        102| 85.75|2023-01-18 00:00:00|
|    1004|        101| 220.0|2023-01-20 00:00:00|
|    1005|        105|1250.0|2023-01-22 00:00:00|
|    1006|        104| 65.99|2023-01-25 00:00:00|
|    1007|        103| 430.5|2023-01-28 00:00:00|
|    1008|        102|199.99|2023-02-01 00:00:00|
|    1009|        101|175.25|2023-02-05 00:00:00|
|    1010|        106| 340.0|2023-02-10 00:00:00|
+--------+-----------+------+-------------------+


DataFrame des clients:
+-----------+----------------+-------+--------------------+
|customer_id|            name|country|               email|
+-----------+----------------+-------+--------------------+
|        101|    Marie Dupont| France|marie.dupont@exam...|
|        1

EXERCICE 6 : Agrégation et Groupement

In [12]:
# Charger les données des commandes
orders_df = spark.read.option("header", "true") \
                      .option("inferSchema", "true") \
                      .csv("orders.csv")

# Charger les données des clients
customers_df = spark.read.option("header", "true") \
                         .option("inferSchema", "true") \
                         .csv("customers.csv")

# Joindre les DataFrames des commandes et des clients
joined_df = orders_df.join(
    customers_df,
    orders_df["customer_id"] == customers_df["customer_id"],
    "inner"
)

# Calculer le montant total des commandes par pays
total_by_country = joined_df.groupBy("country") \
                           .agg(round(spark_sum("amount"), 2).alias("total_amount")) \
                           .orderBy(col("total_amount").desc())

# Afficher les résultats
print("Montant total des commandes par pays (ordre décroissant):")
total_by_country.show()

# Calcul supplémentaire : Pourcentage du montant total par pays
total_amount = joined_df.agg(spark_sum("amount")).collect()[0][0]

# Ajouter une colonne de pourcentage
result_with_percentage = total_by_country.withColumn(
    "percentage", 
    round((col("total_amount") / total_amount) * 100, 2)
)

# Afficher les résultats avec pourcentage
print("\nRépartition des montants par pays avec pourcentage:")
result_with_percentage.show()

Montant total des commandes par pays (ordre décroissant):
+-------+------------+
|country|total_amount|
+-------+------------+
|Germany|      1250.0|
| France|      831.49|
|     UK|      730.49|
|  Spain|       65.99|
+-------+------------+


Répartition des montants par pays avec pourcentage:
+-------+------------+----------+
|country|total_amount|percentage|
+-------+------------+----------+
|Germany|      1250.0|     43.43|
| France|      831.49|     28.89|
|     UK|      730.49|     25.38|
|  Spain|       65.99|      2.29|
+-------+------------+----------+



EXERCICE 7 : Gestion des Valeurs Manquantes

In [13]:
# Charger les données avec valeurs manquantes
df = spark.read.option("header", "true") \
              .option("inferSchema", "true") \
              .csv("missing_data.csv")

# Afficher le DataFrame original
print("DataFrame original:")
df.show()

# 1. ANALYSER LES VALEURS MANQUANTES

# Compter le nombre de valeurs manquantes par colonne
print("\nNombre de valeurs manquantes par colonne:")
df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in df.columns]).show()

# Calculer le pourcentage de valeurs manquantes par colonne
print("\nPourcentage de valeurs manquantes par colonne:")
num_rows = df.count()
for column in df.columns:
    null_count = df.filter(col(column).isNull()).count()
    percentage = (null_count / num_rows) * 100
    print(f"{column}: {percentage:.2f}%")

# 2. DIFFÉRENTES STRATÉGIES POUR GÉRER LES VALEURS MANQUANTES

# A. Supprimer les lignes avec des valeurs manquantes
df_drop_na = df.na.drop()
print("\nDataFrame après suppression des lignes avec valeurs manquantes:")
df_drop_na.show()
print(f"Nombre de lignes restantes: {df_drop_na.count()} sur {num_rows} initialement")

# B. Supprimer les lignes avec au moins X valeurs manquantes
df_drop_thresh = df.na.drop(thresh=4)  # Au moins 4 valeurs non-nulles
print("\nDataFrame après suppression des lignes avec moins de 4 valeurs non-nulles:")
df_drop_thresh.show()

# C. Supprimer les lignes avec des valeurs manquantes seulement dans certaines colonnes
df_drop_subset = df.na.drop(subset=["name", "age"])
print("\nDataFrame après suppression des lignes avec valeurs manquantes dans 'name' ou 'age':")
df_drop_subset.show()

# D. Remplacer les valeurs manquantes par une valeur constante
df_fill_constant = df.na.fill({
    "name": "Inconnu",
    "department": "Non spécifié",
    "experience_years": 0
})
print("\nDataFrame après remplacement de certaines valeurs manquantes par des constantes:")
df_fill_constant.show()

# E. Remplacer les valeurs numériques manquantes par la moyenne
# Calculer la moyenne pour les colonnes numériques
avg_values = {}
for column in ["age", "salary", "experience_years"]:
    try:
        avg = df.agg({column: "avg"}).collect()[0][0]
        if avg is not None:
            avg_values[column] = avg
    except:
        pass

print("\nValeurs moyennes calculées:")
for col, val in avg_values.items():
    print(f"{col}: {val:.2f}")

# Appliquer les moyennes aux valeurs manquantes
df_fill_mean = df.na.fill(avg_values)
print("\nDataFrame après remplacement des valeurs numériques manquantes par la moyenne:")
df_fill_mean.show()

# F. Utiliser l'imputation ML pour les valeurs numériques manquantes
print("\nUtilisation de l'imputation ML:")
# Sélectionner uniquement les colonnes numériques
numeric_cols = ["age", "salary", "experience_years"]
numeric_df = df.select("id", *numeric_cols)

# Configurer l'imputer
imputer = Imputer(
    inputCols=numeric_cols,
    outputCols=[f"{col}_imputed" for col in numeric_cols]
).setStrategy("mean")  # Peut aussi être "median" ou "mode"

# Appliquer l'imputation
imputed_df = imputer.fit(numeric_df).transform(numeric_df)
print("DataFrame après imputation ML des valeurs numériques:")
imputed_df.show()

# G. Stratégie combinée : approche complète
# 1. Supprimer les lignes où trop de valeurs sont manquantes
# 2. Remplacer les valeurs textuelles manquantes par des constantes
# 3. Remplacer les valeurs numériques manquantes par la moyenne

df_combined = df.na.drop(thresh=3)  # Au moins 3 valeurs non-nulles
df_combined = df_combined.na.fill({
    "name": "Inconnu",
    "department": "Non spécifié"
})
df_combined = df_combined.na.fill(avg_values)  # Remplir avec les moyennes calculées précédemment

print("\nDataFrame après application d'une stratégie combinée:")
df_combined.show()

DataFrame original:
+---+---------------+----+------+----------+----------------+
| id|           name| age|salary|department|experience_years|
+---+---------------+----+------+----------+----------------+
|  1|    Jean Dupont|  35| 55000|        IT|            10.0|
|  2|   Marie Martin|  28|  NULL| Marketing|             5.0|
|  3|  Pierre Durand|NULL| 48000|   Finance|            NULL|
|  4|Sophie Lefebvre|  42| 62000|      NULL|            12.0|
|  5|           NULL|  31| 44000|        HR|             7.0|
|  6|     Marc Petit|  45|  NULL|        IT|            15.0|
|  7|  Claire Moreau|  33| 51000| Marketing|            NULL|
|  8|   Thomas Leroy|NULL| 38000|   Finance|             4.0|
|  9|     Julie Roux|  29|  NULL|        HR|            NULL|
| 10|   Paul Bernard|  39| 58000|      NULL|            11.0|
+---+---------------+----+------+----------+----------------+


Nombre de valeurs manquantes par colonne:
+---+----+---+------+----------+----------------+
| id|name|age|sala

EXERCICE 8 : Partitionnement et Optimisation


In [14]:
# Fonction utilitaire pour mesurer les performances
def time_execution(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

# 1. GÉNÉRATION D'UN GRAND DATAFRAME (1 million de lignes)
print("Génération d'un DataFrame volumineux...")

# Méthode 1: Utiliser range pour créer un grand DataFrame
start_time = time.time()
df_large = spark.range(0, 1000000) \
    .withColumn("random1", rand() * 100) \
    .withColumn("random2", rand() * 1000) \
    .withColumn("group", expr("CAST(id % 100 AS INT)")) \
    .withColumn("text", expr("CONCAT('text_', CAST(id AS STRING))"))
end_time = time.time()

print(f"DataFrame généré en {end_time - start_time:.2f} secondes")
print(f"Nombre de lignes: {df_large.count()}")
print(f"Schéma du DataFrame:")
df_large.printSchema()

# 2. VÉRIFICATION DU NOMBRE DE PARTITIONS PAR DÉFAUT
num_partitions_default = df_large.rdd.getNumPartitions()
print(f"\nNombre de partitions par défaut: {num_partitions_default}")

# 3. BENCHMARK AVEC DIFFÉRENTS NOMBRES DE PARTITIONS
partition_sizes = [1, 2, 4, 8, 16, 32, 64]

# Opérations à tester
operations = {
    "count": lambda df: df.count(),
    "filter": lambda df: df.filter(col("random1") > 50).count(),
    "group_by": lambda df: df.groupBy("group").count().collect(),
    "sort": lambda df: df.sort("random1").limit(100).collect(),
    "join": lambda df: df.join(df.select("id", "random1").limit(1000), "id").limit(100).collect()
}

# Stocker les résultats
results = []

print("\n=========== TESTS DE PERFORMANCE ===========")
print("Opération | Nb partitions | Durée (s)")
print("----------------------------------------")

for op_name, op_func in operations.items():
    print(f"\n--- Opération: {op_name} ---")
    
    # D'abord tester avec le nombre de partitions par défaut
    df_test = df_large.cache()  # Mise en cache pour éviter de recalculer le dataframe
    df_test.count()  # Force l'évaluation pour mettre en cache
    
    _, default_time = time_execution(op_func, df_test)
    results.append({
        "operation": op_name,
        "partitions": num_partitions_default,
        "time": default_time
    })
    print(f"{op_name} | {num_partitions_default} (défaut) | {default_time:.4f}")
    
    # Tester avec différents nombres de partitions
    for num_parts in partition_sizes:
        if num_parts != num_partitions_default:
            # Repartitionner le DataFrame
            df_repartitioned = df_large.repartition(num_parts).cache()
            df_repartitioned.count()  # Force l'évaluation pour mettre en cache
            
            _, execution_time = time_execution(op_func, df_repartitioned)
            results.append({
                "operation": op_name,
                "partitions": num_parts,
                "time": execution_time
            })
            print(f"{op_name} | {num_parts} | {execution_time:.4f}")
            
            # Libérer la mémoire
            df_repartitioned.unpersist()
    
    # Libérer la mémoire
    df_test.unpersist()

# 4. ANALYSE ET VISUALISATION DES RÉSULTATS
print("\n=========== RÉSUMÉ DES PERFORMANCES ===========")
print("Voici les meilleurs nombres de partitions par opération:")

# Trouver le meilleur nombre de partitions pour chaque opération
best_partitions = {}
for op_name in operations.keys():
    op_results = [r for r in results if r["operation"] == op_name]
    best_result = min(op_results, key=lambda x: x["time"])
    best_partitions[op_name] = best_result
    
    print(f"{op_name}: {best_result['partitions']} partitions " +
          f"({best_result['time']:.4f}s, " +
          f"{(default_time / best_result['time'] - 1) * 100:.1f}% plus rapide que la valeur par défaut)")

print("\n=========== ENSEIGNEMENTS ===========")
print("""
1. Le nombre optimal de partitions dépend de:
   - La nature de l'opération (count, filter, join, etc.)
   - La taille des données
   - Les ressources disponibles (CPU cores)

2. Tendances générales:
   - Opérations de lecture/scan: plus de partitions = meilleur
   - Opérations de shuffle (groupBy, join): trop de partitions peut dégrader les performances
   - Opérations de tri: dépend de la stratégie de partition

3. Bonnes pratiques:
   - Partitionnez selon le nombre de cœurs disponibles
   - Pour les opérations de shuffle, 2-3x le nombre de cœurs
   - Pour les jointures, partitionnez sur la clé de jointure
   - Testez différentes configurations pour votre cas d'usage spécifique
""")

# Nettoyage
df_large.unpersist()

Génération d'un DataFrame volumineux...
DataFrame généré en 0.12 secondes
Nombre de lignes: 1000000
Schéma du DataFrame:
root
 |-- id: long (nullable = false)
 |-- random1: double (nullable = false)
 |-- random2: double (nullable = false)
 |-- group: integer (nullable = true)
 |-- text: string (nullable = false)


Nombre de partitions par défaut: 12

=========== TESTS DE PERFORMANCE ===========
Opération | Nb partitions | Durée (s)
----------------------------------------

--- Opération: count ---
count | 12 (défaut) | 0.1099
count | 1 | 0.0660
count | 2 | 0.0701
count | 4 | 0.0795
count | 8 | 0.0899
count | 16 | 0.1399
count | 32 | 0.2299
count | 64 | 0.3198

--- Opération: filter ---


TypeError: 'str' object is not callable

EXERCICE 9 : Utilisation de Spark SQL

In [92]:
# Example DataFrame creation (you can replace this with your own DataFrame)
data = [
    (1, "Product A", "Electronics", 150.00),
    (2, "Product B", "Electronics", 200.00),
    (3, "Product C", "Furniture", 300.00),
    (4, "Product D", "Furniture", 400.00),
    (5, "Product E", "Electronics", 250.00)
]

columns = ["id", "name", "category", "price"]
df_products = spark.createDataFrame(data, columns)

# Save DataFrame as a temporary view
df_products.createOrReplaceTempView("products")

# Execute SQL queries
# 1. Select all products
all_products = spark.sql("SELECT * FROM products")
print("All Products:")
all_products.show()

# 2. Filter products by category
electronics_products = spark.sql("SELECT * FROM products WHERE category = 'Electronics'")
print("Electronics Products:")
electronics_products.show()

# 3. Calculate average price by category
avg_price_by_category = spark.sql("SELECT category, AVG(price) as avg_price FROM products GROUP BY category")
print("Average Price by Category:")
avg_price_by_category.show()

All Products:
+---+---------+-----------+-----+
| id|     name|   category|price|
+---+---------+-----------+-----+
|  1|Product A|Electronics|150.0|
|  2|Product B|Electronics|200.0|
|  3|Product C|  Furniture|300.0|
|  4|Product D|  Furniture|400.0|
|  5|Product E|Electronics|250.0|
+---+---------+-----------+-----+

Electronics Products:
+---+---------+-----------+-----+
| id|     name|   category|price|
+---+---------+-----------+-----+
|  1|Product A|Electronics|150.0|
|  2|Product B|Electronics|200.0|
|  5|Product E|Electronics|250.0|
+---+---------+-----------+-----+

Average Price by Category:
+-----------+---------+
|   category|avg_price|
+-----------+---------+
|Electronics|    200.0|
|  Furniture|    350.0|
+-----------+---------+



EXERCICE 10 : Implémentation d'une Fonction de Fenêtre

In [ ]:


# Example DataFrame creation (you can replace this with your own DataFrame)
data = [
    ("John Doe", "IT", 60000),
    ("Jane Smith", "HR", 50000),
    ("Robert Johnson", "Finance", 70000),
    ("Maria Garcia", "IT", 80000),
    ("James Brown", "Marketing", 45000),
    ("Emily Davis", "Finance", 60000),
    ("Michael Wilson", "HR", 55000),
    ("Sarah Thompson", "Marketing", 48000)
]

columns = ["name", "department", "salary"]
df_employees = spark.createDataFrame(data, columns)

# Create a window specification
window_spec = Window.partitionBy("department").orderBy(col("salary").desc())

# Calculate the rank of each employee within their department
df_ranked = df_employees.withColumn("rank", rank().over(window_spec))

# Show the results
print("Employee Ranking by Salary within Each Department:")
df_ranked.show()

# Stop SparkSession


AttributeError: 'NoneType' object has no attribute 'sc'